# dynamo query table

In [3]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from dynamodb_json import json_util
from boto3.dynamodb.conditions import Key, Attr
# from tweet_grab

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()

auth = tweepy.OAuthHandler(creds['twitter-api-key'], creds['twitter-secret-key'])
auth.set_access_token(creds['twitter-access-token'], creds['twitter-secret-access'])

api = tweepy.API(auth)

client = boto3.client('dynamodb',
  endpoint_url = "http://localhost:8000",
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "http://localhost:8000", \
            region_name='us-east-2')

tweets = dynamodb.Table("tweets")
print(tweets.item_count)
    
print("imported modules.")

216341
imported modules.


## scan table
https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Scan.html

In [4]:
done = False
start_key = None
scan_kwargs = {}
table_list = []

while not done:
    if start_key:
        scan_kwargs['ExclusiveStartKey'] = start_key
    response = tweets.scan(**scan_kwargs)
    table_list.extend(response.get('Items', []))
    start_key = response.get('LastEvaluatedKey', None)
    done = start_key is None
    print("read data for key {}".format(start_key))

read data for key {'created_at': '2023-04-24T20:50:27Z', 'user': 'javavvitch'}
read data for key {'created_at': '2022-10-08T04:55:20Z', 'user': 'foglake_'}
read data for key {'created_at': '2022-08-07T23:00:01Z', 'user': 'DubyaEraLeft'}
read data for key {'created_at': '2022-02-12T00:23:29Z', 'user': 'chimneybbb'}
read data for key {'created_at': '2022-07-12T17:37:56Z', 'user': 'charleyhorseatx'}
read data for key {'created_at': '2022-06-02T18:31:14Z', 'user': 'Sapwoood'}
read data for key {'created_at': '2022-05-09T17:07:56Z', 'user': 'thisisnaomie'}
read data for key {'created_at': '2022-07-27T16:40:47Z', 'user': 'JoyceManor'}
read data for key {'created_at': '2023-03-15T17:15:32Z', 'user': 'DigitalBlkHippy'}
read data for key {'created_at': '2021-05-13T02:22:08Z', 'user': 'kladzinski'}
read data for key {'created_at': '2023-03-02T14:50:37Z', 'user': 'loige'}
read data for key {'created_at': '2022-03-22T22:08:48Z', 'user': 'hellahammer'}
read data for key {'created_at': '2023-02-25T1

## response items

## pandas

In [5]:
df = pd.DataFrame(table_list)
df.head()

,favorites,created_at,tweet,id,retweets,user,inserted_at
0,2,2017-09-30T01:44:27Z,Two spoons. https://t.co/KXl5ToyRBC,913942585181396998,0,BEEEZZS,2023-04-28T18:40:59Z
1,6,2017-10-04T01:28:06Z,I should be @ the pitbull concert right now. T...,915388023281504256,0,BEEEZZS,2023-04-28T18:40:59Z
2,2,2017-10-11T02:38:36Z,being 24 i guess is looking for shoes with arc...,917942478187237377,0,BEEEZZS,2023-04-28T18:40:59Z
3,1,2017-10-19T22:31:19Z,@melmphs Wendy's!!! Chicken mozzarella yummm,921141740966498304,0,BEEEZZS,2023-04-28T18:40:59Z
4,3,2017-10-21T02:03:50Z,It's kind of crazy how we all know how to play...,921557611249577984,0,BEEEZZS,2023-04-28T18:40:59Z


In [6]:
df.shape

(216341, 7)

In [7]:
len(np.unique(df["user"]))

418

## summary stats

In [8]:
df.groupby(['user']).agg({'created_at': ['min', 'max'], 'tweet': ['count']})

created_at                       tweet
                                  min                   max count
user                                                             
1_800_dumbdummy  2019-02-20T15:57:03Z  2019-10-21T21:46:25Z   591
23Five           2017-02-21T00:07:20Z  2017-02-21T16:13:50Z     2
AOC              2022-10-10T15:05:45Z  2023-04-28T19:48:07Z   597
ASpittel         2022-04-19T14:32:29Z  2023-04-28T18:04:34Z   600
Aliafonzy43      2023-04-24T15:29:23Z  2023-04-28T20:02:00Z   556
...                               ...                   ...   ...
ypurvazha        2022-05-09T10:52:01Z  2023-04-28T20:10:50Z   572
yrinfinitycrush  2022-10-28T15:34:34Z  2023-04-14T16:53:25Z   274
zachrawson26     2014-11-11T01:33:07Z  2023-04-28T03:51:09Z   591
zmnhn            2014-06-25T02:23:50Z  2020-09-25T01:24:01Z   593
zuza_real        2022-12-29T14:41:49Z  2023-04-28T17:31:33Z   591

[418 rows x 3 columns]

## unique users

In [7]:
dynamo_users = list(np.unique(df['user']))
dynamo_users

['BEEEZZS',
 'CaucasianJames',
 'NECROMANClNG',
 'RedDerby',
 'ViviannRutan',
 'akdjsglhaksWOO',
 'gmailgaze',
 'javavvitch',
 'knifeplay_music',
 'sopharela']

## find missing users in dynamo

In [20]:
my_following

['bastard_band',
 'datacoves',
 'punished_cait',
 'pdrmnvd',
 'ChaunceyCC',
 'MerrittBaer',
 'jasminericegirl',
 'artisinalsprite',
 'Sahelanth',
 'emilianbold',
 'loige',
 'halsnew',
 'pinkninjajess',
 'Amy_Henderson__',
 'breathingatlast',
 'ladaw89',
 'dankbees7',
 'allisonavani1',
 'YeetToOrbit',
 'CalMeNoble6',
 'somethin_2b',
 'notrealneverwas',
 'pilemusic',
 'krilliamhmacy',
 'jpegmafia',
 'kittywavvy',
 'elena_lape',
 'esevatojotx',
 'rrossjd',
 'kittyxcate',
 'hahnscratch',
 'MischievousMal',
 'casiebones',
 'DigitalBlkHippy',
 'ennntropy',
 'shanley',
 'AnnieisUndead',
 'mollysoda',
 'Aliafonzy43',
 'Carnage4Life',
 'fixedpointfae',
 'bernhardsson',
 'freshhel',
 'kvlly',
 'seanjtaylor',
 'ecto_fun',
 'colborkmundus',
 'dataandme',
 'robinson_es',
 'dniellechelosky',
 'LittleMissData',
 'beeonaposy',
 'JoannaNewSum41',
 'titsay',
 'ZeroSuitCamus',
 'keribla',
 'vboykis',
 'Pdaddynthehyena',
 'coolgirl0nline',
 'endingwithali',
 'JamieMuma',
 'jsedlacekjr',
 'tagabowPHL',
 'y

In [26]:
subset = [a not in dynamo_users for a in my_following]
missing_users = [x for x, y in zip(my_following, subset) if y]
missing_users

['punished_cait',
 'pdrmnvd',
 'MerrittBaer',
 'jasminericegirl',
 'artisinalsprite',
 'Sahelanth',
 'emilianbold',
 'halsnew',
 'pinkninjajess',
 'Amy_Henderson__',
 'breathingatlast',
 'allisonavani1',
 'YeetToOrbit',
 'CalMeNoble6',
 'notrealneverwas',
 'jpegmafia',
 'elena_lape',
 'esevatojotx',
 'rrossjd',
 'kittyxcate',
 'hahnscratch',
 'casiebones',
 'DigitalBlkHippy',
 'ennntropy',
 'shanley',
 'AnnieisUndead',
 'mollysoda',
 'Aliafonzy43',
 'Carnage4Life',
 'fixedpointfae',
 'bernhardsson',
 'freshhel',
 'kvlly',
 'seanjtaylor',
 'ecto_fun',
 'colborkmundus',
 'dataandme',
 'robinson_es',
 'dniellechelosky',
 'LittleMissData',
 'beeonaposy',
 'JoannaNewSum41',
 'titsay',
 'ZeroSuitCamus',
 'keribla',
 'vboykis',
 'Pdaddynthehyena',
 'coolgirl0nline',
 'endingwithali',
 'JamieMuma',
 'jsedlacekjr',
 'tagabowPHL',
 'yrinfinitycrush',
 'osoosoband',
 'sama',
 'youngtiddy',
 'FranziaMom',
 'pilatesdev',
 'mike_julian',
 'knifeplay_music',
 'joneaux',
 'GergelyOrosz',
 'p8stie',
 '